In [1]:
import scipy as sp
import numpy as np
import pandas as pd
import timeit
import re
import json
import pickle
import fastparquet
import os
os.chdir('/mnt/t48/bighomes-active/sfeng/patentdiffusion/')
seed = 3
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
import datetime
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.iolib.summary2 as summary2

/homedir/eco/sfeng/bigdata/python/miniconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
jr = pd.read_csv("DataStore/2018-07-P3/JTHReg0727/JTH_res_out_0726.csv",index_col=0)
rr = pd.read_csv("DataStore/2018-08/Reg0820/reg_pairs_out_0820.csv",index_col=0)
rr["samp"] = rr["samp"].apply(eval)
rr = pd.concat([jr,rr],axis=0)
# Reset index
rr = rr.reset_index(drop=True)
rr = rr.replace("C(inv_msa_match)[T.True]", "$I(MSA \, Match)$")

In [3]:
rr.columns

Index(['1975-85', '1985-95', '1995-05', '2005-15', 'LKS', 'Model', 'Model Num',
       'index', 'samp'],
      dtype='object')

In [4]:
rr["id"] = list(zip(rr["samp"], rr["Model Num"]))
rr["id"] = rr["id"].astype(str)

# KS dict
ksd = rr[["id", "LKS"]].drop_duplicates()
ksd = dict(zip(ksd["id"], ksd["LKS"]))

# Sample dict
sd = rr[["id", "samp"]].drop_duplicates()
sd = dict(zip(sd["id"], sd["samp"]))

In [5]:
rr.head()

,1975-85,1985-95,1995-05,2005-15,LKS,Model,Model Num,index,samp,id
0,0.0530***,0.0624***,0.0646***,0.0740***,perc_match_10,"Perc Match Targ MSA, Year FE",0,"$I(MSA \, Match)$",JTH Rep,"('JTH Rep', 0)"
1,(0.0016),(0.0011),(0.0009),(0.0012),perc_match_10,"Perc Match Targ MSA, Year FE",0,NaN,JTH Rep,"('JTH Rep', 0)"
2,58647,107358,185154,154619,perc_match_10,"Perc Match Targ MSA, Year FE",0,$N$,JTH Rep,"('JTH Rep', 0)"
3,0.02,0.03,0.03,0.02,perc_match_10,"Perc Match Targ MSA, Year FE",0,Adjusted $R^2$,JTH Rep,"('JTH Rep', 0)"
4,0.0523***,0.0616***,0.0642***,0.0729***,perc_match_10,"Perc Match Targ MSA, PC FE",1,"$I(MSA \, Match)$",JTH Rep,"('JTH Rep', 1)"


### 1. Results with Year FE and PC Controls

- 4 tables: Norm DV, Norm LDA Vecs, Raw DV, Raw LDA Vecs


Samp    | Norm | Raw 
---------|---------|---------
JTH Rep | 3 | 1 
JTH Targ Sim | 7 |5 
Naics Name | 89 |  31 
PC | 89 | 31 | 

In [22]:
repl1 = pickle.load(open("DataStore/2018-07-P3/reg_names_dict.pkl", "rb"))
# repl2 = {k: "\multicolumn{2}{c}{"+v+"}" if len(v)>1 else v for k,v in repl1.items() }

In [178]:
tab_ind = {}

tab_ind["norm", "ldavecs"] = [str(i) for i in [("JTH Rep", 3), ("JTH Targ Sim", 7),\
       (("ldavecs", "naics_name"), 89), (("ldavecs", "primclass"), 89),]]
tab_ind["raw", "ldavecs"] = [str(i) for i in [("JTH Rep", 1), ("JTH Targ Sim", 5),\
       (("ldavecs", "naics_name"), 31), (("ldavecs", "primclass"), 31),]]
tab_ind["norm", "docvecs"] = [str(i) for i in [("JTH Rep", 3), ("JTH Targ Sim", 7),\
       (("docvecs", "naics_name"), 89), (("docvecs", "primclass"), 89),]]
tab_ind["raw", "docvecs"] = [str(i) for i in [("JTH Rep", 3), ("JTH Targ Sim", 7),\
       (("docvecs", "naics_name"), 31), (("docvecs", "primclass"), 31),]]

#### 1.1 Results by year

In [185]:
# Results by year
v = "norm"
dm = "docvecs"
ygs = ["index", "1975-85", "1985-95", "1995-05"]
final_cols = ["", "1975-85", "1985-95", "1995-05"]
tabp = {}
for v,dm in tab_ind.keys():
    
    for i in tab_ind[v,dm]:
        mod = rr.loc[(rr["id"] == i), ["Model", "Model Num", "samp"]]\
        .drop_duplicates().values[0]
        
        tab = rr.loc[(rr["id"] == i), ygs].fillna("").set_index("index")
    
        # Add other columns
        tab.loc["Year FE"] = [True]*len(tab.columns)
        tab.loc["PC FE"] = [True]*len(tab.columns)

    #     # Pandas tables
        tab = tab.replace(repl1).reset_index()
        tab.columns = final_cols
        
        tabp[v,dm,mod[0],mod[2]] = tab
tab_f = tabp      

In [188]:
for k,v in tab_f.items():
    if (k[0],k[1])==('norm', 'docvecs'):
        print("\n"+str(k)+"\n"+"\n")
        ncols = len(v.columns)
        with pd.option_context("max_colwidth", 1000):
            print(v.to_latex(index=False,escape=False, column_format="lccc"))


('norm', 'docvecs', 'N Perc Match Targ MSA, PC FE', 'JTH Rep')


\begin{tabular}{lccc}
\toprule
                   &    1975-85 &    1985-95 &    1995-05 \\
\midrule
 $I(MSA \, Match)$ &  0.2416*** &  0.2846*** &  0.2966*** \\
                   &   (0.0073) &   (0.0051) &   (0.0041) \\
               $N$ &      58647 &     107358 &     185154 \\
    Adjusted $R^2$ &       0.03 &       0.05 &       0.05 \\
           Year FE &       True &       True &       True \\
             PC FE &       True &       True &       True \\
\bottomrule
\end{tabular}


('norm', 'docvecs', 'N DocVecs - 10 yrs, PC FE', 'JTH Targ Sim')


\begin{tabular}{lccc}
\toprule
                   &    1975-85 &   1985-95 &   1995-05 \\
\midrule
 $I(MSA \, Match)$ &  0.0444*** &    0.0158 &    0.0064 \\
                   &   (0.0145) &  (0.0097) &  (0.0066) \\
               $N$ &      38541 &     69612 &    122217 \\
    Adjusted $R^2$ &       0.03 &      0.03 &      0.04 \\
           Year FE &       True &    

In [190]:
# Results by year - include 2005-15 for similarity
v = "norm"
dm = "docvecs"
ygs = ["index", "1975-85", "1985-95", "1995-05", "2005-15"]
final_cols = ["", "1975-85", "1985-95", "1995-05", "2005-15"]
tabp = {}
for v,dm in tab_ind.keys():
    
    for i in tab_ind[v,dm]:
        mod = rr.loc[(rr["id"] == i), ["Model", "Model Num", "samp"]]\
        .drop_duplicates().values[0]
        
        tab = rr.loc[(rr["id"] == i), ygs].fillna("").set_index("index")
    
        # Add other columns
        tab.loc["Year FE"] = [True]*len(tab.columns)
        tab.loc["PC FE"] = [True]*len(tab.columns)

    #     # Pandas tables
        tab = tab.replace(repl1).reset_index()
        tab.columns = final_cols
        
        tabp[v,dm,mod[0],mod[2]] = tab
tab_f2 = tabp      

In [191]:
for k,v in tab_f2.items():
    if (k[0],k[1])==('norm', 'docvecs'):
        print(k)
        display(v)

('norm', 'docvecs', 'N Perc Match Targ MSA, PC FE', 'JTH Rep')


,,1975-85,1985-95,1995-05,2005-15
0,"$I(MSA \, Match)$",0.2416***,0.2846***,0.2966***,0.3365***
1,,(0.0073),(0.0051),(0.0041),(0.0054)
2,$N$,58647,107358,185154,154619
3,Adjusted $R^2$,0.03,0.05,0.05,0.05
4,Year FE,True,True,True,True
5,PC FE,True,True,True,True


('norm', 'docvecs', 'N DocVecs - 10 yrs, PC FE', 'JTH Targ Sim')


,,1975-85,1985-95,1995-05,2005-15
0,"$I(MSA \, Match)$",0.0444***,0.0158,0.0064,0.0178*
1,,(0.0145),(0.0097),(0.0066),(0.0108)
2,$N$,38541,69612,122217,52710
3,Adjusted $R^2$,0.03,0.03,0.04,0.04
4,Year FE,True,True,True,True
5,PC FE,True,True,True,True


('norm', 'docvecs', 'N PC FE - Mn', ('docvecs', 'naics_name'))


,,1975-85,1985-95,1995-05,2005-15
0,"$I(MSA \, Match)$",0.0323***,0.0605***,0.0610***,0.0571***
1,,(0.0053),(0.0043),(0.0034),(0.0030)
2,$N$,192841,281222,437685,569252
3,Adjusted $R^2$,0.07,0.07,0.08,0.06
4,Year FE,True,True,True,True
5,PC FE,True,True,True,True


('norm', 'docvecs', 'N PC FE - Mn', ('docvecs', 'primclass'))


,,1975-85,1985-95,1995-05,2005-15
0,"$I(MSA \, Match)$",0.0527***,0.0798***,0.0787***,0.0573***
1,,(0.0064),(0.0050),(0.0038),(0.0032)
2,$N$,170882,252174,401623,529686
3,Adjusted $R^2$,0.07,0.07,0.08,0.07
4,Year FE,True,True,True,True
5,PC FE,True,True,True,True


In [192]:
for k,v in tab_f2.items():
    if (k[0],k[1])==('norm', 'docvecs'):
        print("\n"+str(k)+"\n"+"\n")
        ncols = len(v.columns)
        with pd.option_context("max_colwidth", 1000):
            print(v.to_latex(index=False,escape=False, column_format="lccc"))


('norm', 'docvecs', 'N Perc Match Targ MSA, PC FE', 'JTH Rep')


\begin{tabular}{lccc}
\toprule
                   &    1975-85 &    1985-95 &    1995-05 &    2005-15 \\
\midrule
 $I(MSA \, Match)$ &  0.2416*** &  0.2846*** &  0.2966*** &  0.3365*** \\
                   &   (0.0073) &   (0.0051) &   (0.0041) &   (0.0054) \\
               $N$ &      58647 &     107358 &     185154 &     154619 \\
    Adjusted $R^2$ &       0.03 &       0.05 &       0.05 &       0.05 \\
           Year FE &       True &       True &       True &       True \\
             PC FE &       True &       True &       True &       True \\
\bottomrule
\end{tabular}


('norm', 'docvecs', 'N DocVecs - 10 yrs, PC FE', 'JTH Targ Sim')


\begin{tabular}{lccc}
\toprule
                   &    1975-85 &   1985-95 &   1995-05 &   2005-15 \\
\midrule
 $I(MSA \, Match)$ &  0.0444*** &    0.0158 &    0.0064 &   0.0178* \\
                   &   (0.0145) &  (0.0097) &  (0.0066) &  (0.0108) \\
               $N$ &      38

#### 1.1 Results by equation

In [98]:
v = "norm"
dm = "docvecs"
ygs = ["1975-85", "1985-95", "1995-05"]
tabp = {}
for v,dm in tab_ind.keys():
    for yg in ygs:
        rr_t = rr.loc[rr["id"].isin(tab_ind[v,dm])]
        # Drop 2005-15
        rr_t = rr_t.drop("2005-15",1)
        
        for c in tab_ind[v,dm]:
            tab = tab
        # Add other columns
        tab.loc["Year FE"] = [True]*len(tab.columns)
        tab.loc["PC FE"] = [True]*len(tab.columns)

        # Save index
        ind_ord = tab.index.tolist()

        # Add KS, Sample rows, Model Numbers
        KS = [str(ksd[i]) for i in rg.groups.keys()]
#         KS[1::2] = [np.nan]*len(KS[1::2])
        tab.loc["Spillover"] = KS
        Sample = [str(sd[i]) for i in rg.groups.keys()]
#         Sample[1::2] = [np.nan]*len(Sample[1::2])
        tab.loc["Sample"] = Sample
        tab.loc["Model"] = ["({0})".format(i+1) for i in range(len(tab.columns))]
        # Reorder
        tab = tab.reindex(["Sample", "Spillover", "Model"]+ind_ord).reset_index()
        # Create gaps
        tab = tab.fillna("")
        tab.columns = [""]*len(tab.columns)
        
        # Pandas tables
        tab1 = tab.replace(repl1)
        tabp[v,dm,yg] = tab1
tab_yr = tabp      

In [93]:
# for k,v in tabp.items():
#     print(k)
#     display(v)

In [26]:
def conv_latex(l, title, ncols = 4):
    l = ' '.join(l.split())
#     l = l.replace("\\toprule \\\ \\midrule ",\
#                   "\\toprule \multicolumn{"+str(ncols)+"}{c}{"+title+"} \\\ \hline ")
    l = l.replace(" & &", " & ")
    l = l.replace(" & \\\ ", " \\\ ")
    l = l.replace("({0}) \\".format(ncols), "({0}) \\\ \midrule  ".format(ncols))
    return l

for k,v in tabp.items():
    print("\n"+str(k)+"\n"+"\n")
    ncols = len(v.columns)
    with pd.option_context("max_colwidth", 1000):
        print(conv_latex(v.to_latex(index=False,escape=False, column_format="lcccc"),
                        title=k[-1],
                        ncols=ncols))


('norm', 'ldavecs', '1975-85')


\begin{tabular}{lcccc} \toprule \\ \midrule Sample & JTH Cite & JTH Cite Sim & NAICS & Primclass \\ Spillover & $pct\,cites\,in\,MSA_{T}$ & $\overline{sim_{DV}(T,j)}$ & $sim_{LDA}(i,j)$ & $sim_{LDA}(i,j)$ \\ & (1) & (2) & (3) & (4) \\ $I(MSA \, Match)$ & 0.2416*** & 0.0444*** & 0.0465*** & 0.0566*** \\ & (0.0073) & (0.0145) & (0.0050) & (0.0057) \\ $N$ & 58647 & 38541 & 193173 & 171099 \\ Adjusted $R^2$ & 0.03 & 0.03 & 0.11 & 0.18 \\ Year FE & True & True & True & True \\ PC FE & True & True & True & True \\ \bottomrule \end{tabular}

('norm', 'ldavecs', '1985-95')


\begin{tabular}{lcccc} \toprule \\ \midrule Sample & JTH Cite & JTH Cite Sim & NAICS & Primclass \\ Spillover & $pct\,cites\,in\,MSA_{T}$ & $\overline{sim_{DV}(T,j)}$ & $sim_{LDA}(i,j)$ & $sim_{LDA}(i,j)$ \\ & (1) & (2) & (3) & (4) \\ $I(MSA \, Match)$ & 0.2846*** & 0.0158 & 0.0621*** & 0.0647*** \\ & (0.0051) & (0.0097) & (0.0041) & (0.0045) \\ $N$ & 107358 & 69612 & 281625 & 252478 \\ Ad

### 2. NAICS Results with Field Sim

- 4 tables for each vectorization: Norm FE, Norm No FE, Raw FE, Raw No FE


Samp    | Norm PC FE | Norm All FE | Raw PC FE | Raw All FE
---------|---------|---------|---------|---------
PC | 100 | 108 | 42 |50
PC Int | 102 | 110 | 44 | 52
PC MSA | 101 | 109 | 43 | 51
PC MSA Int | 103 | 111 | 45 | 53

In [145]:
repl1 = pickle.load(open("DataStore/2018-07-P3/reg_names_dict.pkl", "rb"))
# repl2 = {k: "\multicolumn{2}{c}{"+v+"}" if len(v)>1 else v for k,v in repl1.items() }

tab_ind = {}

for dm in ["ldavecs", "docvecs"]:
    nf = [100,102,101,103]
    tab_ind["norm", dm, "PC FE"] = ["(('{1}', 'naics_name'), {0})".format(i,dm) for i in nf] 
    nf = [108,110,109,111]
    tab_ind["norm", dm, "All FE"] = ["(('{1}', 'naics_name'), {0})".format(i,dm) for i in nf]
    nf = [42,44,43,45]
    tab_ind["raw", dm, "PC FE"] = ["(('{1}', 'naics_name'), {0})".format(i,dm) for i in nf] 
    nf = [50,52,51,53]
    tab_ind["raw", dm, "All FE"] = ["(('{1}', 'naics_name'), {0})".format(i,dm) for i in nf] 
# Add names for standard errors
rr2 = rr.copy()
se_index = rr2[[yg, "index"]].loc[rr2["index"].isnull()].index
above_var = rr2.loc[se_index-1, "index"].tolist()
rr2.loc[se_index, "index"] = ["(s.e.) "+v for v in above_var]    

#### 2.1 Results by year

In [175]:
ygs = ["index", "1985-95", "1995-05", "2005-15"]
final_cols = ["", "1985-95", "1995-05", "2005-15"]
tab_no_c = {}
tab_c = {}
no_c_ind = ["C(common_est_inv)[T.1.0]", "C(common_pat_inv)[T.True]", "C(lawyer_match)[T.True]"]
no_c_ind = no_c_ind+["(s.e.) "+c for c in no_c_ind]
for v in ["norm", "raw"]:
    for dm in ["docvecs", "ldavecs"]:
        for fe in ["PC FE", "All FE"]:
            for i in tab_ind[v,dm,fe]:
                mod = rr2.loc[(rr2["id"] == i), ["Model", "Model Num", "samp"]]\
        .drop_duplicates().values[0]
        
                tab = rr2.loc[(rr2["id"] == i), ygs].fillna("").set_index("index")
                
                # Add other columns
                tab.loc["Year FE"] = [True]*len(tab.columns)
                tab.loc["PC FE"] = [True]*len(tab.columns)

                if fe == "All FE":
                    tab.loc["Inv & Lawyer Match"] = [True]*len(tab.columns)
                else:
                    tab.loc["Inv & Lawyer Match"] = [False]*len(tab.columns)
                    
                # Create gaps
                tab = tab.fillna("").reset_index()
                
                #1. Create table without controls
                tab2 = tab.copy()
                tab2 = tab2.loc[~(tab2["index"].isin(no_c_ind))]
                # Get rid of se labels
                tab2.loc[tab2["index"].apply(lambda x: "(s.e.)" in x), "index"] = ""
                tab2.columns = final_cols
                tab2 = tab2.replace(repl1)
                
                tab_no_c[v,dm,fe,mod[0],mod[2]] = tab2
                
                #2. Original tables with controls
                # Get rid of se labels
                tab.loc[tab["index"].apply(lambda x: "(s.e.)" in x), "index"] = ""
                tab.columns = final_cols

                # Pandas tables
                tab = tab.replace(repl1)
                tab_c[v,dm,fe,mod[0],mod[2]] = tab
tab_c_yr = tab_c
tab_no_c_yr = tab_no_c

In [184]:
for k,v in tab_no_c_yr.items():
    if (k[0],k[1])==('norm', 'docvecs'):
        print(k)
        display(v)

('norm', 'docvecs', 'PC FE', 'N PC FE - Sim PC - Mn', ('docvecs', 'naics_name'))


,,1985-95,1995-05,2005-15
0,"$I(MSA \, Match)$",0.0389***,0.0352***,0.0319***
1,,(0.0042),(0.0033),(0.0030)
2,"$sim_{DV}(pc_{i}, pc_{j})$",0.2460***,0.2427***,0.2497***
3,,(0.0024),(0.0019),(0.0018)
4,$N$,281018,437485,569100
5,Adjusted $R^2$,0.11,0.12,0.09
6,Year FE,True,True,True
7,PC FE,True,True,True
8,Inv & Lawyer Match,False,False,False


('norm', 'docvecs', 'PC FE', 'N PC FE - Int PC - Mn', ('docvecs', 'naics_name'))


,,1985-95,1995-05,2005-15
0,"$I(MSA \, Match)$",0.0422***,0.0360***,0.0261***
1,,(0.0043),(0.0033),(0.0030)
2,"$I_{MSA} * sim_{DV}(pc_{i}, pc_{j})$",0.0259***,0.0264***,0.0264***
3,,(0.0041),(0.0033),(0.0033)
4,"$sim_{DV}(pc_{i}, pc_{j})$",0.2392***,0.2355***,0.2423***
5,,(0.0026),(0.0021),(0.0021)
6,$N$,281018,437485,569100
7,Adjusted $R^2$,0.11,0.12,0.09
8,Year FE,True,True,True
9,PC FE,True,True,True


('norm', 'docvecs', 'PC FE', 'N PC FE - Sim MSA PC - Mn', ('docvecs', 'naics_name'))


...

,,1985-95,1995-05,2005-15
4,"$I(MSA \, Match)$",0.0401***,0.0319***,0.0219***
5,,(0.0065),(0.0046),(0.0038)
6,"$I_{MSA} * sim_{DV}(pc_{i}, pc_{j})$",0.0134**,0.0196***,0.0240***
7,,(0.0062),(0.0045),(0.0042)
10,"$sim_{DV}(pc_{i}, pc_{j})$",0.2455***,0.2344***,0.2376***
11,,(0.0040),(0.0028),(0.0026)
12,$N$,129662,236607,371516
13,Adjusted $R^2$,0.11,0.13,0.09
14,Year FE,True,True,True
15,PC FE,True,True,True


('norm', 'docvecs', 'All FE', 'N Law FE - Sim MSA PC - Mn', ('docvecs', 'naics_name'))


,,1985-95,1995-05,2005-15
4,"$I(MSA \, Match)$",-0.0183**,-0.0328***,-0.0224***
5,,(0.0072),(0.0049),(0.0039)
8,"$sim_{DV}(pc_{i,MSA_i}, pc_{j,MSA_j})$",0.1866***,0.2075***,0.2036***
9,,(0.0039),(0.0027),(0.0023)
10,$N$,111171,216043,347821
11,Adjusted $R^2$,0.11,0.12,0.08
12,Year FE,True,True,True
13,PC FE,True,True,True
14,Inv & Lawyer Match,True,True,True


('norm', 'docvecs', 'All FE', 'N Law FE - Int MSA PC - Mn', ('docvecs', 'naics_name'))


,,1985-95,1995-05,2005-15
4,"$I(MSA \, Match)$",-0.0176**,-0.0326***,-0.0276***
5,,(0.0072),(0.0049),(0.0042)
6,"$I_{MSA} * sim_{DV}(pc_{i,MSA_i}, pc_{j,MSA_j})$",-0.0131*,-0.0009,0.0106***
7,,(0.0067),(0.0047),(0.0041)
10,"$sim_{DV}(pc_{i,MSA_i}, pc_{j,MSA_j})$",0.1915***,0.2078***,0.2001***
11,,(0.0047),(0.0032),(0.0027)
12,$N$,111171,216043,347821
13,Adjusted $R^2$,0.11,0.12,0.08
14,Year FE,True,True,True
15,PC FE,True,True,True


In [183]:
for k,v in tab_no_c_yr.items():
    if (k[0],k[1])==('norm', 'docvecs'):
        print("\n"+str(k)+"\n"+"\n")
        ncols = len(v.columns)
        with pd.option_context("max_colwidth", 1000):
            print(v.to_latex(index=False,escape=False, column_format="lccc"))


('norm', 'docvecs', 'PC FE', 'N PC FE - Sim PC - Mn', ('docvecs', 'naics_name'))


\begin{tabular}{lccc}
\toprule
                            &    1985-95 &    1995-05 &    2005-15 \\
\midrule
          $I(MSA \, Match)$ &  0.0389*** &  0.0352*** &  0.0319*** \\
                            &   (0.0042) &   (0.0033) &   (0.0030) \\
 $sim_{DV}(pc_{i}, pc_{j})$ &  0.2460*** &  0.2427*** &  0.2497*** \\
                            &   (0.0024) &   (0.0019) &   (0.0018) \\
                        $N$ &     281018 &     437485 &     569100 \\
             Adjusted $R^2$ &       0.11 &       0.12 &       0.09 \\
                    Year FE &       True &       True &       True \\
                      PC FE &       True &       True &       True \\
         Inv & Lawyer Match &      False &      False &      False \\
\bottomrule
\end{tabular}


('norm', 'docvecs', 'PC FE', 'N PC FE - Int PC - Mn', ('docvecs', 'naics_name'))


\begin{tabular}{lccc}
\toprule
                                  

#### 2.2 Results by equation

In [88]:
v = "norm"
dm = "docvecs"
ygs = ["1975-85", "1985-95", "1995-05", "2005-15"]
tab_no_c = {}
tab_c = {}
yg = ygs[1]
no_c_ind = ["C(common_est_inv)[T.1.0]", "C(common_pat_inv)[T.True]", "C(lawyer_match)[T.True]"]
no_c_ind = no_c_ind+["(s.e.) "+c for c in no_c_ind]
for v in ["norm", "raw"]:
    for dm in ["docvecs", "ldavecs"]:
        for fe in ["PC FE", "All FE"]:
            for yg in ygs:
                rr_t = pd.DataFrame()
                for i in tab_ind[v,dm,fe]:
                    rr_t = rr_t.append(rr2.loc[(rr["id"] == i)])
#                 rr2.loc[rr["id"].isin(tab_ind[v,dm,fe])]
                re_ind = rr_t[[yg, "index"]].drop_duplicates("index", keep="first")["index"].tolist()
                # Re-order re-ind
                n_ind = re_ind.index("$N$")
                re_ind = re_ind[:n_ind]+re_ind[n_ind+2:]+re_ind[n_ind:n_ind+2]
                
                # Create table
                tab = pd.DataFrame()
                for i in tab_ind[v,dm,fe]:
                    g = rr_t.loc[(rr_t["id"] == i), [yg, "index"]].set_index("index").reindex(re_ind)
                    tab = pd.concat([tab,g], axis=1)

                # Add other columns
                tab.loc["Year FE"] = [True]*len(tab.columns)
                tab.loc["PC FE"] = [True]*len(tab.columns)

                if fe == "All FE":
                    tab.loc["Inv & Lawyer Match"] = [True]*len(tab.columns)
                else:
                    tab.loc["Inv & Lawyer Match"] = [False]*len(tab.columns)
                    

                # Save index
                ind_ord = tab.index.tolist()
                tab.loc["Model"] = ["({0})".format(i+1) for i in range(len(tab.columns))]
                # Reorder
                tab = tab.reindex(["Model"]+ind_ord).reset_index()
                # Create gaps
                tab = tab.fillna("")
                
                #1. Create table without controls
                tab2 = tab.copy()
                tab2 = tab2.loc[~(tab2["index"].isin(no_c_ind))]
                # Get rid of se labels
                tab2.loc[tab2["index"].apply(lambda x: "(s.e.)" in x), "index"] = ""
                tab2.columns = [""]*len(tab2.columns)
                tab2 = tab2.replace(repl1)
                
                tab_no_c[v,dm,fe,yg] = tab2
                
                #2. Original tables with controls
                # Get rid of se labels
                tab.loc[tab["index"].apply(lambda x: "(s.e.)" in x), "index"] = ""
                tab.columns = [""]*len(tab.columns)

                # Pandas tables
                tab = tab.replace(repl1)
                tab_c[v,dm,fe,yg] = tab


In [94]:
# for k,v in tab_no_c.items():
#     print(k)
#     display(v)

In [95]:
# for k,v in tab_c.items():
#     print(k)
#     display(v)

In [59]:
def conv_latex(l, title, ncols = 8):
    l = ' '.join(l.split())
#     l = l.replace("\\toprule \\\ \\midrule ",\
#                   "\\toprule \multicolumn{"+str(ncols)+"}{c}{"+title+"} \\\ \hline ")
#     l = l.replace(" & &", " & ")
    l = l.replace(" & \\\ ", " \\\ ")
    l = l.replace("(4) \\", "(4) \\\ \midrule  ")
    return l

for k,v in tab_no_c.items():
    print("\n"+str(k)+"\n"+"\n")
    ncols = len(v.columns)
    with pd.option_context("max_colwidth", 1000):
        print(conv_latex(v.to_latex(index=False,escape=False, column_format="lcccc"),
                        title = k[-1],
                        ncols = ncols))


('norm', 'docvecs', 'PC FE', '1975-85')


\begin{tabular}{lcccc} \toprule \\ \midrule & (1) & (2) & (3) & (4) \\ \midrule  \ $I(MSA \, Match)$ & & & \\ & & & \\ $sim_{DV}(pc_{i}, pc_{j})$ & & & \\ & & & \\ $sim_{DV}(pc_{i,MSA_i}, pc_{j,MSA_j})$ & & & \\ & & & \\ $I_{MSA} * sim_{DV}(pc_{i}, pc_{j})$ & & & \\ & & & \\ $I_{MSA} * sim_{DV}(pc_{i,MSA_i}, pc_{j,MSA_j})$ & & & \\ & & & \\ $N$ & & & \\ Adjusted $R^2$ & & & \\ Year FE & True & True & True & True \\ PC FE & True & True & True & True \\ Inv & Lawyer Match & False & False & False & False \\ \bottomrule \end{tabular}

('norm', 'docvecs', 'PC FE', '1985-95')


\begin{tabular}{lcccc} \toprule \\ \midrule & (1) & (2) & (3) & (4) \\ \midrule  \ $I(MSA \, Match)$ & 0.0389*** & -0.0197*** & 0.0422*** & -0.0193*** \\ & (0.0042) & (0.0047) & (0.0043) & (0.0049) \\ $sim_{DV}(pc_{i}, pc_{j})$ & 0.2460*** & & 0.2392*** \\ & (0.0024) & & (0.0026) \\ $sim_{DV}(pc_{i,MSA_i}, pc_{j,MSA_j})$ & & 0.1788*** & & 0.1775*** \\ & & (0.0026) & & (0.0032

In [60]:
for k,v in tab_c.items():
    print("\n"+str(k)+"\n"+"\n")
    ncols = len(v.columns)
    with pd.option_context("max_colwidth", 1000):
        print(conv_latex(v.to_latex(index=False,escape=False, column_format="lcccccccc"),
                        title = k[-1],
                        ncols = ncols))


('norm', 'docvecs', 'PC FE', '1975-85')


\begin{tabular}{lcccccccc} \toprule \\ \midrule Model & (1) & (2) & (3) & (4) \\ \midrule  \ $I(MSA \, Match)$ & & & \\ & & & \\ norm_sim_mean_docvecs_pc & & & \\ & & & \\ norm_sim_mean_docvecs_pc_msa & & & \\ & & & \\ C(inv_msa_match)[T.True]:norm_sim_mean_docvecs_pc & & & \\ & & & \\ C(inv_msa_match)[T.True]:norm_sim_mean_docvecs_pc_msa & & & \\ & & & \\ $N$ & & & \\ Adjusted $R^2$ & & & \\ Year FE & True & True & True & True \\ PC FE & True & True & True & True \\ Inv & Lawyer Match & False & False & False & False \\ \bottomrule \end{tabular}

('norm', 'docvecs', 'PC FE', '1985-95')


\begin{tabular}{lcccccccc} \toprule \\ \midrule Model & (1) & (2) & (3) & (4) \\ \midrule  \ $I(MSA \, Match)$ & 0.0389*** & -0.0197*** & 0.0422*** & -0.0193*** \\ & (0.0042) & (0.0047) & (0.0043) & (0.0049) \\ norm_sim_mean_docvecs_pc & 0.2460*** & & 0.2392*** \\ & (0.0024) & & (0.0026) \\ norm_sim_mean_docvecs_pc_msa & & 0.1788*** & & 0.1775*** \\ & & (0.002